# 🧠 The Deutsch–Jozsa Algorithm — Clear and Intuitive Summary

The **Deutsch–Jozsa algorithm** determines whether a hidden function  
$f: \{0,1\}^n \rightarrow \{0,1\}$  
is **constant** (same output for all inputs) or **balanced** (outputs $0$ for exactly half the inputs and $1$ for the other half).

The power of the algorithm comes from encoding the function into **quantum phase** and then using **interference** to reveal a global property of the function with a **single oracle query**.

---

## 1. Register Initialization

We use:
- $n$ **input qubits**
- $1$ **target (ancilla) qubit**

The input register starts in the state $|0\rangle^{\otimes n}$.  
The target qubit is prepared in $|1\rangle$ using an $X$ gate.

$$
|\psi_0\rangle = |0\rangle^{\otimes n} \otimes |1\rangle
$$

---

## 2. Superposition and State Preparation

We apply:
- $H^{\otimes n}$ to the input qubits  
- $H$ to the target qubit  

This creates an equal superposition over **all possible inputs** and prepares the target qubit in the **minus state** $|-\rangle$.

$$
|\psi_1\rangle
=
\left(
\frac{1}{\sqrt{2^n}}
\sum_{x \in \{0,1\}^n} |x\rangle
\right)
\otimes
\left(
\frac{|0\rangle - |1\rangle}{\sqrt{2}}
\right)
$$

At this point, every input $x$ is present simultaneously with equal weight.

---

## 3. Oracle Transformation ($U_f$)

The oracle is defined as:

$$
U_f |x\rangle |y\rangle = |x\rangle |y \oplus f(x)\rangle
$$

Because the target qubit is in the $|-\rangle$ state, the oracle does **not store** $f(x)$ in the target qubit.  
Instead, the function value appears as a **phase factor** on the input state. This phenomenon is known as **phase kickback**.

### Effect of the Oracle

$$
|\psi_2\rangle
=
\frac{1}{\sqrt{2^n}}
\sum_{x=0}^{2^n-1}
(-1)^{f(x)} |x\rangle \otimes |-\rangle
$$

🔑 **Important:**  
After this step, the target qubit is irrelevant.  
All information about the function is now encoded in the **relative signs** ($\pm$) of the amplitudes of $|x\rangle$.

---

## 4. Interference via Final Hadamard

At this stage, the amplitudes differ **only by phase**, which cannot be observed directly by measurement.  
We therefore apply $H^{\otimes n}$ to the input register to convert **phase information into amplitude information**.

The Hadamard transform acts as:

$$
H^{\otimes n} |x\rangle
=
\frac{1}{\sqrt{2^n}}
\sum_{z \in \{0,1\}^n}
(-1)^{x \cdot z} |z\rangle
$$

Applying this to the full superposition gives:

$$
|\psi_3\rangle
=
\frac{1}{2^n}
\sum_{z=0}^{2^n-1}
\sum_{x=0}^{2^n-1}
(-1)^{f(x) + x \cdot z}
|z\rangle
$$

Here, $x \cdot z$ denotes the **bitwise dot product modulo 2**.

🧠 **Physical meaning:**  
The Hadamard gate causes amplitudes from different $x$ values to **add or cancel** for each output state $|z\rangle$.  
This interference pattern depends entirely on whether the phases $(-1)^{f(x)}$ are uniform or balanced.


**Why this Hadamard behaves differently from the first one:**  
The first $H^{\otimes n}$ acts on $|0\rangle^{\otimes n}$ and simply creates a uniform superposition with no phase differences.  
This second $H^{\otimes n}$ acts on a superposition that already contains **phase information** introduced by the oracle. As a result, amplitudes now **interfere**, causing them to add constructively or cancel out depending on whether $f(x)$ is constant or balanced.

From another perspective, this Hadamard performs a **change of basis**: it maps the state from the phase (Fourier) basis back to the computational basis. In this process, each output basis state $|z\rangle$ collects contributions from *all* input states $|x\rangle$, weighted by the factor $(-1)^{x \cdot z}$.  

The bit string $z$ labels the measurement outcomes. For most values of $z$, the contributions partially cancel. However, for $z = 0$, all phase factors from $x \cdot z$ vanish, so the amplitude depends only on the sum of phases $(-1)^{f(x)}$. This is why the algorithm checks specifically whether $|0\rangle^{\otimes n}$ appears or not.


---

## 5. Measurement and Conclusion


We measure the **input qubits** and examine the probability of obtaining:

$$
|0\rangle^{\otimes n}
$$

For $z = 0$, we have $x \cdot z = 0$, so the amplitude is:

$$
\text{Amplitude of } |0\rangle^{\otimes n}
=
\frac{1}{2^n}
\sum_{x=0}^{2^n-1}
(-1)^{f(x)}
$$


### 🔹 Now the Interference Result (Why Measurement Works)

After applying the final $H^{\otimes n}$, the amplitude of the state  
$|0\rangle^{\otimes n}$ is

$$
\frac{1}{2^n} \sum_x (-1)^{f(x)}
$$

Let us now examine what this sum actually does.

---

### Case 1: $f(x)$ is Constant

This means **every input gives the same function value**.

Either:
- $f(x) = 0$ for all $x$  → every term is $+1$,  
or
- $f(x) = 1$ for all $x$  → every term is $-1$.

So the sum becomes:

$$
(+1) + (+1) + \cdots + (+1) = 2^n
$$

or

$$
(-1) + (-1) + \cdots + (-1) = -2^n
$$

Dividing by $2^n$, the amplitude is:

$$
\pm 1
$$

📌 **All amplitudes add constructively.**  
📌 **All probability goes to** $|0\rangle^{\otimes n}$.

$$
\Rightarrow |0\rangle^{\otimes n} \text{ is measured with certainty}
$$

---

### Case 2: $f(x)$ is Balanced

This means:
- Half of the inputs give $f(x)=0$ → contribute $+1$
- Half of the inputs give $f(x)=1$ → contribute $-1$

So the sum becomes:

$$
(+1) + \cdots + (+1) + (-1) + \cdots + (-1)
$$

There are equal numbers of $+1$ and $-1$, so:

$$
(+1) + (-1) = 0
$$

and the entire sum cancels:

$$
\sum_x (-1)^{f(x)} = 0
$$

Dividing by $2^n$, the amplitude is:

$$
0
$$

📌 **Perfect destructive interference occurs.**  
📌 **The state** $|0\rangle^{\otimes n}$ **disappears completely.**

$$
\Rightarrow |0\rangle^{\otimes n} \text{ is never measured}
$$

---

### 🔹 Why This Works for Any $n$

Although the number of terms grows exponentially, the logic does not change:

- **Constant function** → all signs are the same → amplitudes add  
- **Balanced function** → equal $+$ and $-$ → amplitudes cancel  

This is why a **single measurement** is enough to distinguish the two cases.


---

## ✅ Final Result — What Measurement Tells Us

After applying the final Hadamard and measuring the input qubits:

### Case 1: Measured state is $|0\rangle^{\otimes n}$

- This means **all amplitudes added constructively**.  
- Only happens if the function is **constant** (all $f(x)$ are the same).  
- **Conclusion:** The function is **constant**.

### Case 2: Measured state is any $|z\rangle \neq |0\rangle^{\otimes n}$

- This means **destructive interference canceled the amplitude of $|0\rangle^{\otimes n}$**.  
- Only happens if the function is **balanced** (half $+1$, half $-1$).  
- **Conclusion:** The function is **balanced**.

---

### 🔑 Key Takeaway

1. The **oracle** encodes the function into **phase differences** of the input states.  
2. The **final Hadamard** turns these phase differences into measurable amplitudes (interference).  
3. **Measuring the input qubits** reveals the global property of the function:  
   - $|0\rangle^{\otimes n}$ → constant  
   - Any other state → balanced

✅ This is why the Deutsch–Jozsa algorithm can determine the property of the function with **just one oracle query**.



In [ ]:
import cirq
import numpy as np

"""
Implementation of the Deutsch-Jozsa Algorithm using Cirq.

The Deutsch-Jozsa problem involves determining whether a given function f: {0,1}^n -> {0,1} is constant or balanced.
A constant function returns the same output for all inputs, while a balanced function returns 0 for half of the inputs and 1 for the other half.

Table 1. Constant Function Truth Table
Input Qubit State |x⟩ | f(x)
00                0
01                0
10                0
11                0

Table 2. Balanced Function Truth Table
Input Qubit State |x⟩ | f(x)
00                0
01                1
10                1
11                0

The Deutsch-Jozsa algorithm determines whether a given function is constant or balanced using a quantum oracle.

The oracle function implements the oracle for the balanced function as well as for the constant function. 
For the constant function, we do not apply any transformation, and hence it implements f(x) = 0. 
For the balanced function of two input qubits, we implement it using a CNOT transform on the target qubit based on the 2 input qubit states in succession. 
For the computational basis state |x1 x2⟩, the balanced function can be written as f(x) = x0 ⊕ x1.
"""


def oracle(data_reg, y_reg, circuit, is_balanced=True):
    """
    Implements the Deutsch-Jozsa oracle on the circuit.
    
    :param data_reg: List of input qubits
    :param y_reg: Target qubit
    :param circuit: Cirq circuit
    :param is_balanced: Whether the oracle is balanced (True) or constant (False)
    :return: Updated circuit with oracle applied
    """
    if is_balanced:
        # Balanced function: f(x) = x0 ⊕ x1
        # Apply CNOT from each data qubit to the target qubit
        circuit.append([cirq.CNOT(data_reg[0], y_reg),
                        cirq.CNOT(data_reg[1], y_reg)])
    # For constant function, do nothing
    return circuit


def deutsch_jozsa(domain_size: int,
                   func_type_to_simulate: str = "balanced",
                   copies: int = 1000):
    """
    Executes the Deutsch-Jozsa algorithm.
    
    :param domain_size: Number of input states (2^n)
    :param func_type_to_simulate: 'balanced' or 'constant'
    :param copies: Number of repetitions for simulation
    :return: None, prints circuit and results
    """
    # Define number of input qubits
    reqd_num_qubits = int(np.ceil(np.log2(domain_size)))
    
    # Define the input qubits
    # Input qubits are at indices 0 to reqd_num_qubits - 1
    data_reg = [cirq.LineQubit(c) for c in range(reqd_num_qubits)]
    
    # Define the target qubit 
    # Target qubit is at index reqd_num_qubits
    y_reg = cirq.LineQubit(reqd_num_qubits)
    
    # Define the circuit
    circuit = cirq.Circuit()
    
    # Prepare equal superposition state for input qubits
    circuit.append(cirq.H(q) for q in data_reg)
    
    # Prepare minus superposition state for target qubit
    circuit.append(cirq.X(y_reg))
    circuit.append(cirq.H(y_reg))
    
    # Determine oracle type
    # Balanced or Constant
    is_balanced = func_type_to_simulate.lower() == 'balanced'
    
    # Apply the oracle
    circuit = oracle(data_reg, y_reg, circuit, is_balanced=is_balanced)
    
    # Apply Hadamard transform on input qubits again
    # This step helps to interfere the amplitudes
    circuit.append(cirq.H(q) for q in data_reg)
    
    # Measure the input qubits
    circuit.append(cirq.measure(*data_reg, key='z'))
    
    # Print circuit
    print("Circuit Diagram Follows:")
    print(circuit)
    
    # Simulate
    sim = cirq.Simulator()
    result = sim.run(circuit, repetitions=copies)
    
    # Print histogram
    print("Measurement Results:")
    print(result.histogram(key='z'))


if __name__ == '__main__':
    print("Execute Deutsch-Jozsa for a Balanced Function of Domain size 4")
    deutsch_jozsa(domain_size=4, func_type_to_simulate='balanced', copies=1000)
    
    print("\nExecute Deutsch-Jozsa for a Constant Function of Domain size 4")
    deutsch_jozsa(domain_size=4, func_type_to_simulate='constant', copies=1000)


Execute Deutsch-Jozsa for a Balanced Function of Domain size 4
Circuit Diagram Follows:
0: ───H───────@───H───────M('z')───
              │           │
1: ───H───────┼───@───H───M────────
              │   │
2: ───X───H───X───X────────────────
Measurement Results:
Counter({3: 1000})

Execute Deutsch-Jozsa for a Constant Function of Domain size 4
Circuit Diagram Follows:
0: ───H───H───M('z')───
              │
1: ───H───H───M────────

2: ───X───H────────────
Measurement Results:
Counter({0: 1000})


In [ ]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
import math

"""
Implementation of the Deutsch-Jozsa Algorithm using Qiskit.

The Deutsch-Jozsa problem involves determining whether a given function f: {0,1}^n -> {0,1} is constant or balanced.
A constant function returns the same output for all inputs, while a balanced function returns 0 for half of the inputs and 1 for the other half.

Table 1. Constant Function Truth Table
Input Qubit State |x⟩ | f(x)
00                0
01                0
10                0
11                0

Table 2. Balanced Function Truth Table
Input Qubit State |x⟩ | f(x)
00                0
01                1
10                1
11                0

The Deutsch-Jozsa algorithm determines whether a given function is constant or balanced using a quantum oracle.

The oracle function implements the oracle for the balanced function as well as for the constant function. 
For the constant function, we do not apply any transformation, and hence it implements f(x) = 0. 
For the balanced function of two input qubits, we implement it using a CNOT transform on the target qubit based on the 2 input qubit states in succession. 
For the computational basis state |x1 x2⟩, the balanced function can be written as f(x) = x0 ⊕ x1.
"""


def oracle(data_reg, y_reg, circuit, is_balanced=True):
    """
    Implements the Deutsch-Jozsa oracle on the circuit.
    
    :param data_reg: List of input qubit indices
    :param y_reg: Target qubit index
    :param circuit: Qiskit QuantumCircuit
    :param is_balanced: Whether the oracle is balanced (True) or constant (False)
    :return: Updated circuit with oracle applied
    """
    if is_balanced:
        # For XOR function f(x) = x0 ⊕ x1
        # Apply CNOT from each input qubit to target qubit
        circuit.cx(data_reg[0], y_reg)
        circuit.cx(data_reg[1], y_reg)
    # For constant function, do nothing
    return circuit


def deutsch_jozsa(domain_size: int,
                   func_type_to_simulate: str = "balanced",
                   copies: int = 1000):
    """
    Executes the Deutsch-Jozsa algorithm.
    
    :param domain_size: Number of input states (2^n)
    :param func_type_to_simulate: 'balanced' or 'constant'
    :param copies: Number of repetitions for simulation
    :return: None, prints circuit and results
    """
    # Define number of input qubits
    reqd_num_qubits = math.ceil(math.log2(domain_size))
    
    # Define the input qubits indices
    data_reg = list(range(reqd_num_qubits))
    
    # Define the target qubit index
    y_reg = reqd_num_qubits
    
    # Create the circuit with total qubits = input qubits + 1 target qubit
    circuit = QuantumCircuit(reqd_num_qubits + 1, reqd_num_qubits)
    
    # Prepare equal superposition state for input qubits
    for q in data_reg:
        circuit.h(q)
    
    # Prepare minus superposition state for target qubit
    circuit.x(y_reg)
    circuit.h(y_reg)
    
    # Determine oracle type: balanced or constant
    is_balanced = func_type_to_simulate.lower() == 'balanced'
    
    # Apply the oracle
    circuit = oracle(data_reg, y_reg, circuit, is_balanced=is_balanced)
    
    # Apply Hadamard transform on input qubits again
    for q in data_reg:
        circuit.h(q)
    
    # Measure the input qubits
    circuit.measure(data_reg, range(reqd_num_qubits))
    
    # Print circuit diagram
    print("Circuit Diagram Follows:")
    print(circuit.draw('text'))
    
    # Simulate
    simulator = AerSimulator()
    result = simulator.run(circuit, shots=copies).result()
    counts = result.get_counts()
    
    # Print histogram/results
    print("Measurement Results:")
    print(counts)


if __name__ == '__main__':
    print("Execute Deutsch-Jozsa for a Balanced Function of Domain size 4")
    deutsch_jozsa(domain_size=4, func_type_to_simulate='balanced', copies=1000)
    
    print("\nExecute Deutsch-Jozsa for a Constant Function of Domain size 4")
    deutsch_jozsa(domain_size=4, func_type_to_simulate='constant', copies=1000)


Execute Deutsch-Jozsa for a Balanced Function of Domain size 4
Circuit Diagram Follows:
     ┌───┐          ┌───┐     ┌─┐   
q_0: ┤ H ├───────■──┤ H ├─────┤M├───
     ├───┤       │  └───┘┌───┐└╥┘┌─┐
q_1: ┤ H ├───────┼────■──┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐┌─┴─┐└───┘ ║ └╥┘
q_2: ┤ X ├┤ H ├┤ X ├┤ X ├──────╫──╫─
     └───┘└───┘└───┘└───┘      ║  ║ 
c: 2/══════════════════════════╩══╩═
                               0  1 
Measurement Results:
{'11': 1000}

Execute Deutsch-Jozsa for a Constant Function of Domain size 4
Circuit Diagram Follows:
     ┌───┐┌───┐┌─┐   
q_0: ┤ H ├┤ H ├┤M├───
     ├───┤├───┤└╥┘┌─┐
q_1: ┤ H ├┤ H ├─╫─┤M├
     ├───┤├───┤ ║ └╥┘
q_2: ┤ X ├┤ H ├─╫──╫─
     └───┘└───┘ ║  ║ 
c: 2/═══════════╩══╩═
                0  1 
Measurement Results:
{'00': 1000}



## Explanation of Measurement Results in the Deutsch-Jozsa Algorithm

We executed the Deutsch-Jozsa algorithm for **2 input qubits** using Qiskit and obtained the following measurement results:

**Balanced Function Example:**
$Measurement Results: {'11': 1000}$

**Constant Function Example:**
$Measurement Results: {'00': 1000}$

Let's analyze why this happens.

---

### 1️⃣ Initial Setup

- **Input qubits:** $|x_0 x_1\rangle$ initialized in $|0\rangle$  
- **Target qubit:** $|y\rangle$ initialized in $|1\rangle$  
- **Balanced function:** $f(x) = x_0 \oplus x_1$  
- **Constant function:** $f(x) = 0$  

We prepare the target qubit in the **minus state**:

$$
|-\rangle = \frac{|0\rangle - |1\rangle}{\sqrt{2}}
$$

and put the input qubits into **equal superposition** using Hadamards:

$$
H^{\otimes 2} |00\rangle = \frac{1}{2}(|00\rangle + |01\rangle + |10\rangle + |11\rangle)
$$

---

### 2️⃣ Oracle Action

- The oracle flips the target qubit **if $f(x) = 1$**.  
- Since the target is in $|-\rangle$, flipping it **adds a phase $(-1)^{f(x)}$ to the input qubits**:

$$
|x\rangle |-\rangle \xrightarrow{\text{oracle}} (-1)^{f(x)} |x\rangle |-\rangle
$$

For the balanced function $f(x) = x_0 \oplus x_1$:

| $x$   | $f(x)$ | Phase $(-1)^{f(x)}$ |
|-----|------|----------------|
| 00  | 0    | +1             |
| 01  | 1    | -1             |
| 10  | 1    | -1             |
| 11  | 0    | +1             |

Resulting state (input qubits only):

$$
|\psi_{\text{after oracle}}\rangle = \frac{1}{2} (|00\rangle - |01\rangle - |10\rangle + |11\rangle)
$$

---

### 3️⃣ Apply Hadamard Again on Input Qubits

- The second layer of Hadamards causes **interference** among all amplitudes:  

$$
H^{\otimes 2} |x\rangle = \frac{1}{2} \sum_{y \in \{00,01,10,11\}} (-1)^{x \cdot y} |y\rangle
$$

- Combining the oracle phases with the Hadamard transform:

$$
|\psi_{\text{final}}\rangle = \sum_{y} \frac{1}{2} \sum_{x} (-1)^{f(x) + x \cdot y} |y\rangle
$$

### 3️⃣ Compute the amplitude sums

After applying the Hadamard gates on the input qubits again, the final amplitude for each computational basis state $|y_0 y_1\rangle$ is:

$$
\text{amplitude}(y) = \frac{1}{2} \sum_{x \in \{00,01,10,11\}} (-1)^{f(x) + x \cdot y}
$$

where $x \cdot y = x_0 y_0 + x_1 y_1$ (mod 2) and $f(x) = x_0 \oplus x_1$.

---

#### Step-by-step for each $y$:

**1. $y = 00$**

$$
\begin{aligned}
\sum_x (-1)^{f(x) + x \cdot 00} &= (-1)^{f(00)+0} + (-1)^{f(01)+0} + (-1)^{f(10)+0} + (-1)^{f(11)+0} \\
&= (+1) + (-1) + (-1) + (+1) \\
&= 0
\end{aligned}
$$

**2. $y = 01$**

$$
\begin{aligned}
\sum_x (-1)^{f(x) + x \cdot 01} &= (-1)^{0 + 0} + (-1)^{1 + 1} + (-1)^{1 + 0} + (-1)^{0 + 1} \\
&= (+1) + (+1) + (-1) + (-1) \\
&= 0
\end{aligned}
$$

**3. $y = 10$**

$$
\begin{aligned}
\sum_x (-1)^{f(x) + x \cdot 10} &= (-1)^{0 + 0} + (-1)^{1 + 0} + (-1)^{1 + 1} + (-1)^{0 + 1} \\
&= (+1) + (-1) + (+1) + (-1) \\
&= 0
\end{aligned}
$$

**4. $y = 11$**

$$
\begin{aligned}
\sum_x (-1)^{f(x) + x \cdot 11} &= (-1)^{0 + 0} + (-1)^{1 + 1} + (-1)^{1 + 1} + (-1)^{0 + 2} \\
&= (+1) + (+1) + (+1) + (+1) \\
&= 4
\end{aligned}
$$

---

| $y$   | Amplitude sum | Outcome |
|-----|---------------|---------|
| 00  | 0             | canceled |
| 01  | 0             | canceled |
| 10  | 0             | canceled |
| 11  | 4             | survives |

✅ Only $|11\rangle$ has non-zero amplitude → measurement always gives `11`.

---

### 4️⃣ Constant Function Case

- For the constant function $f(x)=0$, all phases are $+1$.  
- After the second Hadamard on input qubits:

$$
|\psi_{\text{final}}\rangle = |00\rangle
$$

✅ Only $|00\rangle$ has non-zero amplitude → measurement always gives `00`.

---

### 5️⃣ Key Takeaways

- Deutsch-Jozsa **does not produce random outcomes**; it deterministically tells whether the function is **constant** or **balanced**.  
- **Measurement outcomes:**
  - Constant → always $|00\rangle$  
  - Balanced → always some **non-zero state** ($|11\rangle$ in our XOR oracle example)  
- This determinism comes from **quantum interference**, which cancels out all amplitudes except the ones indicating the function type.
